In [34]:
import glob
import pandas as pd
import os
import numpy as np
from data_pre import datagenerator
import cv2
from keras.utils import np_utils
from tensorflow.random import set_seed
from tensorflow import keras
import seaborn as sns
from find_next import findesec
from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import pickle


In [ ]:
train=!ls dataset/ #path to the dataset
train

In [20]:

def ANN_chart(y_train,Xp_chart,Xt_chart,y_test):
    y_train_ANN = np_utils.to_categorical(y_train)
    y_test_ANN = np_utils.to_categorical(y_test)
    set_seed(4*28)
    inputs = keras.Input(shape=Xp_chart.shape[1])
    hidden_layer = keras.layers.Dense(128, activation="relu")(inputs)
    output_layer = keras.layers.Dense(19, activation="softmax")(hidden_layer)
    model = keras.Model(inputs=inputs, outputs=output_layer)
    print(model.summary())
    model.compile(optimizer='adam', loss=keras.losses.CategoricalCrossentropy())
    history = model.fit(Xp_chart, y_train_ANN, epochs=500)
    sns.lineplot(x=history.epoch, y=history.history['loss'])
    model.save('model_Ann_4*28_6000_chart')
    y_pred = model.predict(Xt_chart)
    cc=0
    for  i in range(len(y_pred)):
        if np.argmax(y_pred[i]) == np.argmax(y_test_ANN[i]):
            cc += 1
    ACC=100-((cc/len(y_pred))*100)
    print(f'ACC : {ACC}')
    re_list=np.zeros([len(y_pred),6])
    for i in range(len(y_pred)):
        list_acc=findesec.finde_second(y_pred[i])
        for j in range(4):
            re_list[i][j]=list_acc[j] 
        re_list[i][4]=ACC            
        re_list[i][5]=np.argmax(y_test_ANN[i])
    return re_list

In [21]:
def ANN_len(y_train,Xp_len,y_test,Xt_len):
    y_train_ANN = np_utils.to_categorical(y_train)
    y_test_ANN = np_utils.to_categorical(y_test)
    set_seed(4*28)
    inputs = keras.Input(shape=Xp_len.shape[1])
    hidden_layer = keras.layers.Dense(128, activation="relu")(inputs)
    output_layer = keras.layers.Dense(19, activation="softmax")(hidden_layer)
    model = keras.Model(inputs=inputs, outputs=output_layer)
    print(model.summary())
    model.compile(optimizer='adam', loss=keras.losses.CategoricalCrossentropy())
    history = model.fit(Xp_len, y_train_ANN, epochs=500)
    sns.lineplot(x=history.epoch, y=history.history['loss'])
    model.save('model_Ann_4*28_6000_len')
    y_pred = model.predict(Xt_len)
    cc=0
    for  i in range(len(y_pred)):
        if np.argmax(y_pred[i]) == np.argmax(y_test_ANN[i]):
            cc+=1
    ACC=100-((cc/len(y_pred))*100)
    print(f'ACC : {ACC}')
    re_list=np.zeros([len(y_pred),6])
    for i in range(len(y_pred)):
        list_acc=findesec.finde_second(y_pred[i])
        for j in range(4):
            re_list[i][j]=list_acc[j] 
        re_list[i][4]=ACC            
        re_list[i][5]=np.argmax(y_test_ANN[i])
    return re_list


In [22]:
def ANN_P(y_train,Xp_p,y_test,Xt_p):
    y_train_ANN = np_utils.to_categorical(y_train)
    y_test_ANN = np_utils.to_categorical(y_test)
    set_seed(2*28)
    inputs = keras.Input(shape=Xp_p.shape[1])
    hidden_layer = keras.layers.Dense(128, activation="relu")(inputs)
    output_layer = keras.layers.Dense(19, activation="softmax")(hidden_layer)
    model = keras.Model(inputs=inputs, outputs=output_layer)
    print(model.summary())
    model.compile(optimizer='adam', loss=keras.losses.CategoricalCrossentropy())
    history = model.fit(Xp_p, y_train_ANN, epochs=500)
    sns.lineplot(x=history.epoch, y=history.history['loss'])
    model.save('model_Ann_2*28_6000_len')
    y_pred = model.predict(Xt_p)
    cc=0
    for  i in range(len(y_pred)):
        if np.argmax(y_pred[i]) == np.argmax(y_test_ANN[i]):
            cc+=1
    ACC=100-((cc/len(y_pred))*100)
    print(f'ACC : {ACC}')
    re_list=np.zeros([len(y_pred),6])
    for i in range(len(y_pred)):
        list_acc=findesec.finde_second(y_pred[i])
        for j in range(4):
            re_list[i][j]=list_acc[j] 
        re_list[i][4]=ACC            
        re_list[i][5]=np.argmax(y_test_ANN[i])
    return re_list

In [35]:
def SVM_len(y_train,Xp_len,y_test,Xt_len):
    param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf']}#,'poly'
    svc=svm.SVC(probability=True)
    model_svm_len=GridSearchCV(svc,param_grid)
    model_svm_len.fit(Xp_len,y_train)
    pickle.dump(model_svm_len,open('model_svm_4*28_6000_len','wb'))
    y_pred=model_svm_len.predict(Xt_len)
    print(f"The model is {100-accuracy_score(y_pred,y_test)*100}% accurate")
    print(len(y_pred))
    ACC=accuracy_score(y_pred,y_test)*100
    re_list=np.zeros([len(y_pred),4])
    for i in range(len(y_pred)):
        re_list[i][0]=0.99
        re_list[i][1]=y_pred[i]
        re_list[i][2]=ACC            
        re_list[i][3]=y_test[i]
    return re_list


def SVM_chart(y_train,Xp_chart,Xt_chart,y_test):
    param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf']}#,'poly']}
    svc=svm.SVC(probability=True)
    model_svm_chart=GridSearchCV(svc,param_grid)
    model_svm_chart.fit(Xp_chart,y_train)
    y_pred=model_svm_chart.predict(Xt_chart)
    pickle.dump(model_svm_chart,open('model_svm_4*28_6000_chart','wb'))
    y_pred=model_svm_chart.predict(Xt_chart)
    print(f"The model is {100-accuracy_score(y_pred,y_test)*100}% accurate")
    ACC=accuracy_score(y_pred,y_test)*100
    re_list=np.zeros([len(y_pred),4])
    for i in range(len(y_pred)):
        re_list[i][0]=0.99
        re_list[i][1]=y_pred[i]
        re_list[i][2]=ACC            
        re_list[i][3]=y_test[i]
    return re_list

def SVM_p(y_train,Xp_chart,Xt_chart,y_test):
    param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf']}#,'poly']}
    svc=svm.SVC(probability=True)
    model_svm_chart=GridSearchCV(svc,param_grid)
    model_svm_chart.fit(Xp_chart,y_train)
    y_pred=model_svm_chart.predict(Xt_chart)
    pickle.dump(model_svm_chart,open('model_svm_2*28_6000_chart','wb'))
    y_pred=model_svm_chart.predict(Xt_chart)
    print(f"The model is {100-accuracy_score(y_pred,y_test)*100}% accurate")
    ACC=accuracy_score(y_pred,y_test)*100
    re_list=np.zeros([len(y_pred),4])
    for i in range(len(y_pred)):
        re_list[i][0]=0.99
        re_list[i][1]=y_pred[i]
        re_list[i][2]=ACC            
        re_list[i][3]=y_test[i]
    return re_list

In [3]:
x_train_path=[]
x_test_path=[]
y_train=[]
y_test=[]

for i in range(len(train)):
    art=!ls dataset/{train[i]}/
    nm_im=len(art)
    nm_im_train=int(nm_im*0.7)
    nm_im_test=int(nm_im-nm_im_train)
    b=(glob.glob(f"dataset/{train[i]}/*"))

    for j in range(nm_im_train):
        x_train_path.append(b[j])
        y_train.append(train[i])
    
    for j in range( nm_im_train, nm_im ):
        x_test_path.append(b[j])
        y_test.append(train[i])

In [4]:
x_train=[]
for i in range(len(x_train_path)):
    image=cv2.imread(x_train_path[i])
    img=cv2.resize(image, (28, 28))
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    x_train.append(grayscale_image)
x_test=[]
for i in range(len(x_test_path)):
    image=cv2.imread(x_test_path[i])
    img=cv2.resize(image, (28, 28))
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    x_test.append(grayscale_image)

In [6]:
x_train_array=np.zeros([len(x_train),28,28])
x_test_array=np.zeros([len(x_test),28,28])

for i in range(len(x_train)):
    for j in range(28):
        for k in range(28):
            x_train_array[i][j][k]=x_train[i][j][k]
            
for i in range(len(x_test)):
    for j in range(28):
        for k in range(28):
            x_test_array[i][j][k]=x_test[i][j][k]

In [7]:
mode_chart='chart' # this is a mode for glass abstraction
mode_num=4
Xp_chart , Xt_chart = datagenerator.detamake(x_train_array,x_test_array,mode_chart,mode_num)
mode_len='len' # this is a mode for opeauqe abstraction
mode_num=4
Xp_len , Xt_len = datagenerator.detamake(x_train_array,x_test_array,mode_len,mode_num)
mode_p='p' # this is a mode for pick abstraction
mode_num=2
Xp_p , Xt_p = datagenerator.detamake(x_train_array,x_test_array,mode_p,mode_num)

column=["PR_ANN_chart_1","LA_ANN_chart_1","PR_ANN_chart_2","LA_ANN_chart_2","ACC_ANN_chart",
        "PR_ANN_len_1","LA_ANN_len_1","PR_ANN_len_2","LA_ANN_len_2","ACC_ANN_len",
        "PR_SVM_chart_1","LA_SVM_chart_1","ACC_SVM_chart",
        "PR_SVM_len_1","LA_SVM_len_1","ACC_SVM_len",
        "LA_Ture"]

In [12]:
class_dict = {'0':0, '1':1, '2':2, '3':3, '4':4, '5':5,'6':6, '7':7, '8':8, '9':9,
 'add':10, 'dec':11, 'div':12, 'eq':13, 'mul':14, 'sub':15, 'x':16, 'y':17, 'z':18}
y_train_array=np.zeros([len(y_train)])
y_test_array=np.zeros([len(y_test)])

for i in range(len(y_train)):
    y_train_array[i]=class_dict[y_train[i]]

for i in range(len(y_test)):
    y_test_array[i]=class_dict[y_test[i]]

In [33]:
df_total = pd.DataFrame(index=range(len(y_test)),columns=column)

# re_list_ANN_chart=ANN_chart(y_train_array,Xp_chart,Xt_chart,y_test_array)
# re_list_ANN_len=ANN_len(y_train_array,Xp_len,y_test_array,Xt_len)
# re_list_ANN_len=ANN_p(y_train_array,Xp_p,y_test_array,Xt_p)

re_list_SVM_chart=SVM_chart(y_train_array,Xp_chart,Xt_chart,y_test_array)
re_list_SVM_len=SVM_len(y_train_array,Xp_len,y_test_array,Xt_len)
re_list_SVM_len=SVM_p(y_train_array,Xp_p,Xt_p,y_test_array)


NameError: name 'pickle' is not defined

In [ ]:
for i in range(len(column)): 
    for j in range(len(y_test)):
        if np.isnan(df_total.loc[j,"LA_Ture"]):
            df_total.loc[j,"LA_Ture"]=re_list_SVM_chart[j][3]
        if column[i] == "PR_ANN_chart_1":
            if df_total.loc[j,"LA_Ture"] == re_list_ANN_chart[j][5]:
                for k in range(5):
                    df_total.loc[j,column[i+k]]=re_list_ANN_chart[j][k]
        if column[i] == "PR_ANN_len_1":
            if df_total.loc[j,"LA_Ture"] == re_list_ANN_len[j][5]:
                for k in range(5):
                    df_total.loc[j,column[i+k]]=re_list_ANN_len[j][k]
        if column[i] == "PR_SVM_chart_1":
            if df_total.loc[j,"LA_Ture"] == re_list_SVM_chart[j][3]:            
                for k in range(3):
                    df_total.loc[j,column[i+k]]=re_list_SVM_chart[j][k]
        if column[i] == "PR_SVM_len_1":
            if df_total.loc[j,"LA_Ture"] == re_list_SVM_len[j][3]:            
                for k in range(3):
                    df_total.loc[j,column[i+k]]=re_list_SVM_len[j][k]
df_total.to_csv("total.csv")

column_2 = ["PR_ANN_chart_1","LA_ANN_chart_1","PR_ANN_chart_2","LA_ANN_chart_2",
            "PR_ANN_len_1","LA_ANN_len_1","PR_ANN_len_2","LA_ANN_len_2",
            "PR_SVM_chart_1","LA_SVM_chart_1","PR_SVM_len_1","LA_SVM_len_1","LA_Ture"]
        
df_sub = pd.DataFrame(index=range(len(y_test)),columns=column_2)
for i in range(len(y_test)):
    df_sub.loc[i,"PR_ANN_chart_1"]=df_total.loc[i,"PR_ANN_chart_1"]
for i in range(len(y_test)):
    df_sub.loc[i,"LA_ANN_chart_1"]=df_total.loc[i,"LA_ANN_chart_1"]
for i in range(len(y_test)):
    df_sub.loc[i,"PR_ANN_chart_2"]=df_total.loc[i,"PR_ANN_chart_2"]
for i in range(len(y_test)):
    df_sub.loc[i,"LA_ANN_chart_2"]=df_total.loc[i,"LA_ANN_chart_2"]
for i in range(len(y_test)):
    df_sub.loc[i,"PR_ANN_len_1"]=df_total.loc[i,"PR_ANN_len_1"]
for i in range(len(y_test)):
    df_sub.loc[i,"LA_ANN_len_1"]=df_total.loc[i,"LA_ANN_len_1"]
for i in range(len(y_test)):
    df_sub.loc[i,"PR_ANN_len_2"]=df_total.loc[i,"PR_ANN_len_2"]    
for i in range(len(y_test)):
    df_sub.loc[i,"LA_ANN_len_2"]=df_total.loc[i,"LA_ANN_len_2"]  
for i in range(len(y_test)):
    df_sub.loc[i,"PR_SVM_chart_1"]=df_total.loc[i,"PR_SVM_chart_1"] 
for i in range(len(y_test)):
    df_sub.loc[i,"LA_SVM_chart_1"]=df_total.loc[i,"LA_SVM_chart_1"] 
for i in range(len(y_test)):
    df_sub.loc[i,"PR_SVM_len_1"]=df_total.loc[i,"PR_SVM_len_1"]     
for i in range(len(y_test)):
    df_sub.loc[i,"LA_SVM_len_1"]=df_total.loc[i,"LA_SVM_len_1"] 
for i in range(len(y_test)):
    df_sub.loc[i,"LA_Ture"]=df_total.loc[i,"LA_Ture"]   

In [ ]:
df_sub.to_csv("sub_cnn_cnn_svm.csv", header=False, index=False)

In [ ]:
# train anfis model same as theother applications(mnist and emnist)